<a href="https://colab.research.google.com/github/MuhammadIrzam447/MultiModel/blob/master/Valid_43.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
import os
from PIL import Image
import torch
from torchvision import datasets, transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
import torchvision
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import UnidentifiedImageError
from sklearn.metrics import classification_report, roc_auc_score

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cuda


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

# Loading Validation Dataset and Preprocessing

In [ ]:
# save_dir = '/content/Model/Fused_ResNetmodel_Hatefull_Experiment#5'
# load_path = os.path.join(save_dir, 'model.pth')

# # Create an instance of the ResNet model
# resnet = torchvision.models.resnet101(pretrained=False)
# resnet.fc = nn.Linear(2048, 2) # Choose the number of output classses as per your model

# # Load the saved model parameters
# # resnet.load_state_dict(torch.load(load_path, map_location=torch.device('cpu')))
# resnet.load_state_dict(torch.load(load_path))

# # Set the model to evaluation mode and respective device
# resnet.eval()
# resnet.to(device)

In [3]:
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
class ValidationDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.dataset = datasets.ImageFolder(data_dir, transform=val_transform)
        self.classes = sorted(os.listdir(data_dir))
        self.class_lengths = self._compute_class_lengths()

    def _compute_class_lengths(self):
        class_lengths = {cls: 0 for cls in self.classes}

        for cls in self.classes:
            cls_dir = os.path.join(self.data_dir, cls)
            if os.path.isdir(cls_dir):
                class_lengths[cls] = len(os.listdir(cls_dir))

        return class_lengths

    def __getitem__(self, index):
        image, label = self.dataset[index]
        return image, label

    def __len__(self):
        return len(self.dataset)

In [5]:
valPath = "/content/Dataset(s)/fused-ferramenta-val"
val_dataset = ValidationDataset(valPath)

In [6]:
batch_size = 32
validation_data_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [7]:
print("Number of samples:", len(val_dataset))
print("Number of classes:", len(val_dataset.classes))

Number of samples: 21869
Number of classes: 52


In [8]:
num_batches = len(validation_data_loader)
print("Number of batches:", num_batches)

Number of batches: 684


# Validation

In [9]:
epoch = 1
save_dir = '/content/Model/Models-Train-10'
while epoch < 27:
    model_name = str(epoch+1) + "_model.pth"
    load_path = os.path.join(save_dir, model_name)

    # Create an instance of the ResNet model
    resnet = torchvision.models.resnet101(pretrained=False)
    resnet.fc = nn.Linear(2048, 52) # Choose the number of output classses as per your model

    # Load the saved model parameters
    # resnet.load_state_dict(torch.load(load_path, map_location=torch.device('cpu')))
    resnet.load_state_dict(torch.load(load_path))

    # Set the model to evaluation mode and respective device
    resnet.eval()
    resnet.to(device)

    # resnet.eval()

    predicted_classes = []
    actual_labels = []

    with torch.no_grad():
        for images, labels in validation_data_loader:
            # Move the images and labels to the GPU if available
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = resnet(images)

            # Get the predicted labels
            _, predicted_label = torch.max(outputs, 1)

            # Store the predicted and true labels
            predicted_classes.extend(predicted_label.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())

        accuracy = accuracy_score(actual_labels, predicted_classes)
        precision = precision_score(actual_labels, predicted_classes, average='weighted')
        recall = recall_score(actual_labels, predicted_classes, average='weighted')
        f1 = f1_score(actual_labels, predicted_classes, average='weighted')

        print("Result with model " + model_name + " : ")
        print("=========================================")
        print("Accuracy:", accuracy)
        print("Precision:", precision)
        print("Recall:", recall)
        print("F1-score:", f1)
        print(classification_report(actual_labels, predicted_classes))

        predicted_classes = np.array(predicted_classes)
        actual_labels = np.array(actual_labels)

        # Calculate the AUROC score
        # auroc = roc_auc_score(actual_labels, predicted_classes)
        # print("AUROC:", auroc)
        epoch = epoch + 1


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Result with model 2_model.pth : 
Accuracy: 0.9570625085737803
Precision: 0.9571563447215355
Recall: 0.9570625085737803
F1-score: 0.9568593710671013
              precision    recall  f1-score   support

           0       0.80      0.63      0.70        59
           1       0.97      0.98      0.98      1879
           2       0.97      0.99      0.98       345
           3       1.00      0.98      0.99        48
           4       0.94      1.00      0.97       132
           5       0.91      0.93      0.92       176
           6       0.98      0.99      0.98      1069
           7       0.96      0.95      0.96       148
           8       0.97      0.97      0.97       663
           9       0.99      0.99      0.99       642
          10       0.94      0.96      0.95       136
          11       0.94      0.97      0.96        35
          12       0.82      0.96      0.89       112
          13       0.96      0.97      0.97       842
          14       0.97      0.98      0.

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Result with model 3_model.pth : 
Accuracy: 0.9591202158306278
Precision: 0.9590558958734685
Recall: 0.9591202158306278
F1-score: 0.9586568199527834
              precision    recall  f1-score   support

           0       0.79      0.69      0.74        59
           1       0.98      0.98      0.98      1879
           2       0.97      0.99      0.98       345
           3       1.00      1.00      1.00        48
           4       0.95      1.00      0.97       132
           5       0.92      0.89      0.90       176
           6       0.97      0.99      0.98      1069
           7       0.97      0.96      0.96       148
           8       0.97      0.99      0.98       663
           9       0.99      1.00      0.99       642
          10       0.95      0.96      0.95       136
          11       0.81      0.97      0.88        35
          12       0.88      0.96      0.91       112
          13       0.97      0.97      0.97       842
          14       0.98      0.99      0.

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


...

Result with model 25_model.pth : 
Accuracy: 0.9617723718505647
Precision: 0.9616950190448025
Recall: 0.9617723718505647
F1-score: 0.9615530584859653
              precision    recall  f1-score   support

           0       0.90      0.59      0.71        59
           1       0.98      0.98      0.98      1879
           2       0.98      0.99      0.98       345
           3       0.98      1.00      0.99        48
           4       0.96      0.99      0.97       132
           5       0.94      0.94      0.94       176
           6       0.98      0.99      0.99      1069
           7       0.97      0.93      0.95       148
           8       0.97      0.98      0.98       663
           9       0.99      1.00      0.99       642
          10       0.94      0.95      0.95       136
          11       0.94      0.91      0.93        35
          12       0.88      0.96      0.92       112
          13       0.98      0.97      0.97       842
          14       0.98      0.99      0

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Result with model 26_model.pth : 
Accuracy: 0.9617723718505647
Precision: 0.9616444640655052
Recall: 0.9617723718505647
F1-score: 0.9614636200538211
              precision    recall  f1-score   support

           0       0.88      0.64      0.75        59
           1       0.98      0.98      0.98      1879
           2       0.98      0.98      0.98       345
           3       1.00      1.00      1.00        48
           4       0.96      1.00      0.98       132
           5       0.93      0.95      0.94       176
           6       0.98      0.99      0.99      1069
           7       0.96      0.94      0.95       148
           8       0.97      0.98      0.98       663
           9       0.98      1.00      0.99       642
          10       0.93      0.96      0.94       136
          11       0.92      0.97      0.94        35
          12       0.88      0.96      0.92       112
          13       0.97      0.98      0.98       842
          14       0.98      0.99      0

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Result with model 27_model.pth : 
Accuracy: 0.9604005670126663
Precision: 0.9603657685846897
Recall: 0.9604005670126663
F1-score: 0.9602362042289522
              precision    recall  f1-score   support

           0       0.74      0.71      0.72        59
           1       0.98      0.98      0.98      1879
           2       0.97      0.98      0.98       345
           3       0.98      1.00      0.99        48
           4       0.94      1.00      0.97       132
           5       0.92      0.94      0.93       176
           6       0.98      0.99      0.98      1069
           7       0.94      0.92      0.93       148
           8       0.97      0.98      0.98       663
           9       0.99      1.00      0.99       642
          10       0.94      0.96      0.95       136
          11       0.91      0.91      0.91        35
          12       0.85      0.95      0.90       112
          13       0.97      0.98      0.97       842
          14       0.97      0.99      0

In [ ]:
accuracy = accuracy_score(actual_labels, predicted_classes)
precision = precision_score(actual_labels, predicted_classes, average='weighted')
recall = recall_score(actual_labels, predicted_classes, average='weighted')
f1 = f1_score(actual_labels, predicted_classes, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

In [ ]:
print(classification_report(actual_labels, predicted_classes))

In [ ]:
labels = ['Accuracy', 'Precision', 'Recall', 'F1-score']
values = [accuracy, precision, recall, f1]

x = np.arange(len(labels))

fig, ax = plt.subplots(figsize=(5, 4))
bars = ax.bar(x, values)

# labels, title, and legend
ax.set_xlabel('Metrics')
ax.set_ylabel('Score')
ax.set_title('Model Performance Metrics')
ax.set_xticks(x)
ax.set_xticklabels(labels)

# scores on top of each bar
for i, bar in enumerate(bars):
    score = values[i]
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{score:.2f}', ha='center', va='bottom')

plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(actual_labels, predicted_classes)
ConfusionMatrixDisplay(cm).plot()

In [ ]:
# Count the occurrences of each label in predicted and true labels
predicted_counts = np.bincount(predicted_classes)
true_counts = np.bincount(actual_labels)

# Get the unique labels
labels = np.unique(np.concatenate((predicted_classes, actual_labels)))

# Set the x-axis range
x = np.arange(len(labels))

# Set the width of the bars
width = 0.35

# Plot the predicted and true label counts
fig, ax = plt.subplots(figsize=(5,5))
ax.bar(x - width/2, predicted_counts, width, label='Predicted Labels')
ax.bar(x + width/2, true_counts, width, label='True Labels')

# Add labels, title, and legend
ax.set_xlabel('Labels')
ax.set_ylabel('Count')
ax.set_title('Distribution of Predicted and True Labels')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

plt.show()